# Air Quality Data Format Usage Examples

This notebook demonstrates how to use the converted air quality data formats:
- Parquet files (fast, compressed, columnar storage)
- DuckDB database (SQL query support)
- Data loader utilities (convenient Python interface)

**Author:** Claude Code  
**Date:** 2025-10-13

## Setup and Imports

In [ ]:
# Import required libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'src' / 'main' / 'python'))

# Import our custom utilities
from utils.data_loader import AirQualityDataLoader, load_air_quality_data, query_air_quality

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Imports successful")

## 1. Basic Data Loading from Parquet

Parquet files provide fast, compressed storage with efficient column-wise access.

In [ ]:
# Initialize data loader
loader = AirQualityDataLoader()

# Get basic information about the dataset
min_date, max_date = loader.get_date_range()
print(f"Data available from {min_date} to {max_date}")

# Get list of monitoring stations
stations = loader.get_station_list()
print(f"\nTotal monitoring stations: {len(stations)}")
print(f"\nSample stations:")
print(stations[['sitename', 'county']].head(10))

## 2. Loading Data with Filters

Efficiently load only the data you need using filters.

In [ ]:
# Load one month of data for Taipei City
df_taipei = loader.load_parquet(
    start_date='2024-08-01',
    end_date='2024-08-31',
    counties=['Taipei City']
)

print(f"Loaded {len(df_taipei):,} rows")
print(f"\nDataset info:")
print(df_taipei.info())
print(f"\nSample data:")
df_taipei.head()

In [ ]:
# Load data for specific stations
df_stations = loader.load_parquet(
    start_date='2024-08-01',
    end_date='2024-08-31',
    stations=['Zhongshan', 'Wanhua', 'Songshan'],
    columns=['date', 'sitename', 'aqi', 'pm2.5', 'pm10', 'o3']
)

print(f"Loaded {len(df_stations):,} rows for selected stations")
df_stations.head()

## 3. Loading Data by Year

Data is partitioned by year for efficient access.

In [ ]:
# Load all data for 2024
df_2024 = loader.load_by_year(2024)

print(f"2024 data: {len(df_2024):,} rows")
print(f"Date range: {df_2024['date'].min()} to {df_2024['date'].max()}")
print(f"\nCounties covered:")
print(df_2024['county'].value_counts().head(10))

## 4. SQL Queries with DuckDB

Use SQL for complex analytical queries.

In [ ]:
# Query: Average AQI by county for 2024
df_county_avg = loader.query_db("""
    SELECT 
        county,
        COUNT(*) as measurement_count,
        AVG(aqi) as avg_aqi,
        MAX(aqi) as max_aqi,
        AVG(pm2_5) as avg_pm25
    FROM air_quality
    WHERE year = 2024
    GROUP BY county
    ORDER BY avg_aqi DESC
""")

print("Average AQI by County (2024):")
df_county_avg

In [ ]:
# Query: Top 10 stations with highest average PM2.5
df_top_pm25 = loader.query_db("""
    SELECT 
        sitename,
        county,
        AVG(pm2_5) as avg_pm25,
        MAX(pm2_5) as max_pm25,
        COUNT(*) as measurements
    FROM air_quality
    WHERE year = 2024 AND pm2_5 IS NOT NULL
    GROUP BY sitename, county
    ORDER BY avg_pm25 DESC
    LIMIT 10
""")

print("Top 10 Stations by Average PM2.5 (2024):")
df_top_pm25

In [ ]:
# Query: Monthly trends using the pre-built view
df_monthly = loader.query_db("""
    SELECT 
        year,
        month,
        county,
        avg_aqi,
        avg_pm25
    FROM monthly_summary
    WHERE year = 2024 AND county = 'Taipei City'
    ORDER BY month
""")

print("Monthly Trends for Taipei City (2024):")
df_monthly

## 5. Summary Statistics

Get quick summary statistics using convenience functions.

In [ ]:
# Get summary statistics for all counties
summary_all = loader.get_summary_stats()
print("Summary Statistics by County:")
summary_all

In [ ]:
# Get summary for a specific county
summary_taipei = loader.get_summary_stats(county='Taipei City')
print("Summary Statistics for Taipei City:")
summary_taipei

## 6. Time Series Analysis

Analyze temporal patterns in air quality.

In [ ]:
# Load recent data for time series analysis
df_recent = load_air_quality_data(
    start_date='2024-08-01',
    end_date='2024-08-31',
    county='Taipei City'
)

# Calculate daily averages
df_daily = df_recent.groupby(df_recent['date'].dt.date).agg({
    'aqi': 'mean',
    'pm2.5': 'mean',
    'pm10': 'mean',
    'o3': 'mean'
}).reset_index()

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].plot(df_daily['date'], df_daily['aqi'])
axes[0, 0].set_title('Daily Average AQI')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('AQI')
axes[0, 0].tick_params(axis='x', rotation=45)

axes[0, 1].plot(df_daily['date'], df_daily['pm2.5'], color='orange')
axes[0, 1].set_title('Daily Average PM2.5')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('PM2.5 (μg/m³)')
axes[0, 1].tick_params(axis='x', rotation=45)

axes[1, 0].plot(df_daily['date'], df_daily['pm10'], color='red')
axes[1, 0].set_title('Daily Average PM10')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('PM10 (μg/m³)')
axes[1, 0].tick_params(axis='x', rotation=45)

axes[1, 1].plot(df_daily['date'], df_daily['o3'], color='green')
axes[1, 1].set_title('Daily Average O3')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('O3 (ppb)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✓ Time series plots generated")

## 7. Spatial Analysis

Analyze air quality patterns across different locations.

In [ ]:
# Get average AQI by station with geographic coordinates
df_spatial = loader.query_db("""
    SELECT 
        sitename,
        county,
        AVG(longitude) as longitude,
        AVG(latitude) as latitude,
        AVG(aqi) as avg_aqi,
        AVG(pm2_5) as avg_pm25
    FROM air_quality
    WHERE year = 2024 AND longitude IS NOT NULL
    GROUP BY sitename, county
""")

# Create scatter plot colored by AQI
plt.figure(figsize=(12, 10))
scatter = plt.scatter(
    df_spatial['longitude'],
    df_spatial['latitude'],
    c=df_spatial['avg_aqi'],
    s=100,
    cmap='RdYlGn_r',
    alpha=0.6,
    edgecolors='black'
)
plt.colorbar(scatter, label='Average AQI')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Average AQI by Monitoring Station (2024)')
plt.grid(True, alpha=0.3)
plt.show()

print("✓ Spatial distribution plot generated")

## 8. Pollution Event Analysis

Identify and analyze high pollution events.

In [ ]:
# Query high pollution events using the pre-built view
df_high_pollution = loader.query_db("""
    SELECT *
    FROM high_pollution_events
    WHERE year = 2024
    LIMIT 20
""")

print("Top 20 High Pollution Events (AQI > 100) in 2024:")
df_high_pollution[['date', 'sitename', 'county', 'aqi', 'pollutant', 'status']]

In [ ]:
# Analyze pollution status distribution
df_status = loader.query_db("""
    SELECT 
        status,
        COUNT(*) as count,
        AVG(aqi) as avg_aqi
    FROM air_quality
    WHERE year = 2024 AND status IS NOT NULL
    GROUP BY status
    ORDER BY avg_aqi
""")

# Create bar chart
plt.figure(figsize=(10, 6))
plt.bar(df_status['status'], df_status['count'])
plt.xlabel('Air Quality Status')
plt.ylabel('Number of Measurements')
plt.title('Distribution of Air Quality Status (2024)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nAir Quality Status Distribution:")
print(df_status)

## 9. Performance Comparison

Compare query performance between different data formats.

In [ ]:
import time

# Benchmark: Load one month of data from Parquet
start = time.time()
df_parquet = loader.load_parquet(
    start_date='2024-08-01',
    end_date='2024-08-31'
)
parquet_time = time.time() - start

# Benchmark: Same query using DuckDB SQL
start = time.time()
df_duckdb = loader.query_db("""
    SELECT *
    FROM air_quality
    WHERE date BETWEEN '2024-08-01' AND '2024-08-31'
""")
duckdb_time = time.time() - start

print("Performance Comparison:")
print(f"Parquet load time: {parquet_time:.2f} seconds ({len(df_parquet):,} rows)")
print(f"DuckDB query time: {duckdb_time:.2f} seconds ({len(df_duckdb):,} rows)")
print(f"\nSpeedup factor: {max(parquet_time, duckdb_time) / min(parquet_time, duckdb_time):.2f}x")

## 10. Cleanup

Close database connections when done.

In [ ]:
# Close database connection
loader.close()
print("✓ Database connection closed")

## Summary

This notebook demonstrated:

1. **Loading data from Parquet files** with filters
2. **SQL queries with DuckDB** for complex analytics
3. **Convenience functions** for common operations
4. **Time series analysis** of air quality trends
5. **Spatial analysis** of pollution patterns
6. **Event detection** for high pollution episodes
7. **Performance comparisons** between formats

### Key Takeaways

- **Parquet format** provides fast, efficient storage (70-80% smaller than CSV)
- **DuckDB** enables powerful SQL analytics without loading full dataset
- **Data loader utilities** provide convenient Python interface
- **Partitioning by year** enables efficient data access
- **Pre-built views** simplify common query patterns

### Next Steps

- Explore more advanced time series analysis (seasonality, trends)
- Build machine learning models for air quality prediction
- Create interactive dashboards with Plotly/Dash
- Integrate with external data sources (weather, traffic)
- Develop real-time monitoring and alerting systems